In [1]:
import pygame
import pygame_menu
import pygame.freetype
import sys
import random
import math
import numpy as np

from gym import Env
from gym.spaces import Discrete, Box

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

pygame.init()

pygame 2.0.1 (SDL 2.0.14, Python 3.6.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


(8, 0)

In [ ]:
class Custom_OpenAI_Env(Env):
    def __init__(self, screen_width, screen_height, screen, surface, game):
        # Actions we can take; left, right, shoot, no_action
        self.action_space = Discrete(4)
        # Array of observation, dont know how to implement this as image based input.
        self.observation_space = Box(low=0, high=255, 
                                            shape=(screen_height, screen_width, 3), dtype=np.uint8)
        # Initial state should be image of the first frame of the game
        self.state = self.get_state()
        self.start_state = self.state
        
        #Initialise pygame
        self.game = game# Space_Invaders(screen_width, screen_height, screen, surface)

    def step(self, action):
        # Apply action
        self.game.execute_action(action)
        
        #Call the update loop before getting the state
        self.game.update()
        self.state = self.game.get_current_state()

        # Calculate reward
        #Calculated as follows: if player is still alive, +0.001 (perhaps), if player is dead, -1
        #increasing function dependant on how many aliens are left (the less there are the higher
        #the value from value of 0 - 1 with 0 being all still alive, 1 being all dead, game over)
        reward = self.game.calculate_reward()
        
        # Check if shower is done
        #if function equals 1, return done
        done = self.game.done
            
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info
    def render(self):
        #Put render loop in here
        self.game.render()
    def reset(self):
        #Restart the game
        self.state = self.start_state
        self.game.reset()
    
#Super class for developing custom environments more easily    
class Training_Game():
    def __init__(self, width, height, screen, surface, mode):
        pass
    def get_current_state(self):
        pass
    def get_reward(self):
        pass
    def execute_action(self):
        pass
    def render(self):
        pass
    def update(self):
        pass
    def reset(self):
        pass

In [2]:

#Core Space invaders game mode.
class Space_Invaders(Training_Game):
    def __init__(self, width, height, screen, surface, mode):
        #Pygame initialisation variables
        GAME_FONT = pygame.freetype.Font("COMIC.TTF", 24)
        self.width = width
        self.height = height
        self.screen = pygame.display.set_mode((width, height), pygame.DOUBLEBUF)
        self.surface = surface     
        self.clock = pygame.time.Clock()
        
        #Game state check
        self.done = False
        self.lost = False
        self.training_mode = mode
        
        #Enemy general variables
        self.enemySpeed = 0.25
        self.enemyShotTimer = random.uniform(0.5, 10.0)
        self.enemyRockets = []
        self.potentialShooters = []
        self.aliens = []
        self.generator = Generator(self)
        self.max_aliens = len(self.aliens)
        
        #Misc
        self.player = Invaders_Player(self, width/2, height-25)
        self.all_sprites_list = pygame.sprite.Group()
        
        #If in a normal game, start the standard game loop, otherwise this will be controlled
        #by the space invader env super-class.
        if self.training_mode == False:
            while not self.done:
                self.update()
                self.render()
            
            self.lost = False
            
        def execute_action(self, action):
            if action == 0:
                self.move_left()
            if action == 1:
                self.move_right()
            if action == 2:
                self.shoot()
                
        def move_left(self):
            self.player.sprite.rect.x -=2 if  self.player.sprite.rect.x > 20 else 0
            
        def move_right(self):
            self.player.sprite.rect.x +=2 if self.player.sprite.rect.x < width-20 else 0
            
        def shoot(self):
            self.rockets.append(Rocket(self, self.player.sprite.rect.x+16, self.player.sprite.rect.y))
            self.player.fire_rate = 3.0
            
        def get_current_state(self):
            self.pixel_array = pygame.PixelArray(self.surface)
            print("surface information before: \ndimensions: ", self.pixel_array.ndim, "\nShape: ",
                self.pixel_array.shape, "\n")
            self.pixel_array.close()
            print("surface information after: \ndimensions: ", self.pixel_array.ndim, "\nShape: ",
                self.pixel_array.shape, "\n")
            return self.pixel_array.flatten()
        def calculate_reward(self):
            #Return 1 or -1 to indicate a loss or win terminal state
            if self.lost == True:
                return -1
            if self.done and not self.lost:
                return 1
            #If still alive, calculate a function, clamped between 0-0.01 giving small reward,
            #scaled upon how many aliens are dead in the state to hopefully promote both preserving
            #its own life and killing other aliens.
            aliens_remaining = len(self.aliens)
            aliens_score = ((self.max_aliens - aliens_remaining)/ self.max_aliens)/100
            return aliens_score
            
        def end_state(self, hasWon):
            if hasWon == True and self.training_mode == False:
                GAME_FONT.render_to(self.screen, (40, 350), "Victory", (0, 0, 255))
                GAME_FONT.render_to(self.screen, (40, 300), "Space to Continue.", (0, 0, 255))
                for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                        self.done = True
            elif hasWon == False and self.training_mode == False:
                self.lost = True
                GAME_FONT.render_to(self.screen, (40, 350), "Defeat", (0, 0, 255))
                GAME_FONT.render_to(self.screen, (40, 300), "Space to Continue.", (0, 0, 255))
                for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                        self.done = True
            else:
                self.done = True
        def reset(self):
            #Reset the clock
            self.clock = pygame.time.Clock()
            #Reset the end conditions
            self.done = False
            self.lost = False
            #Reinitialise all of the values containing game objects
            self.enemyShotTimer = random.uniform(0.5, 10.0)
            self.enemyRockets = []
            self.potentialShooters = []
            self.aliens = []
            #Generate a new array of aliens and reinitialise the player
            self.generator = Generator(self)
            self.player = Invaders_Player(self, width/2, height-25)
        def update(self):
             #Check for the victory condition (zero aliens left)
            if len(self.aliens) == 0:
                #If this is a genuine game, show the victory screen and await input
                self.end_state(True)
            
            #If the player is not dead, draw and check collisions
            if not self.lost: 
                self.player.draw(self)
                self.player.checkCollision(self)
            else:
                #If the game is just a normal game, display defeat screen and awat prompt
                self.end_state(False)
            
            #Only check and register input if not a training game
            if self.training_mode == False:
                #Loop through pygame event to check for key presses.
                for event in pygame.event.get():
                    #Only enable leaving the game for non-training builds
                    if event.type == pygame.QUIT and self.training_mode == False:
                        self.done = True
                    #Shoot a bullet if fire rate bounds are satisfied and the space bar is held down
                    if not self.lost and event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                        if self.player.fire_rate == 0.0:
                            self.shoot()

                #Get the current pressed buttons and call move functionality
                pressed = pygame.key.get_pressed()
                if pressed[pygame.K_LEFT]:
                    self.move_left()
                elif pressed[pygame.K_RIGHT]:
                    self.move_right()
                    
            #Control Enemy laser fire
            if self.enemyShotTimer <= 0.0:                    
                #potential rows are 0 - 11 for the standard game
                self.potentialShooters.clear()
                for index in range(0, 11):
                    #Initialise variables for finding the lowest clear alien on each row
                    highest_row = -1
                    current_best = None
                    for alien in self.aliens:
                        if alien.row_index == index:
                            if alien.row > highest_row:
                                highest_row = alien.row
                                current_best = alien
                        if current_best != None:
                            self.potentialShooters.append(current_best)
                    #Pick a random one of the potential shooters and fire    
                    if len(self.enemyRockets) == 0:
                        shooter = random.randrange(0, len(self.potentialShooters))
                        self.potentialShooters[shooter].fire(self)
                        self.enemyShotTimer = random.uniform(2.0, 6.0)
            #Otherwise iterate down the fire cool-down for the aliens.
            else:
                self.enemyShotTimer -= 0.1
            
        def render(self):            
            #Fill the screen background black
            self.screen.fill((0,0,0))
            
            #Render the score text to the screen
            GAME_FONT.render_to(self.screen, (40, 350), str(self.player.score), (0, 0, 255))
            
            #Update the aliens
            for alien in self.aliens:
                #Render all the remaining aliens
                alien.draw(self)
                #If the game is still going, check collisions with the aliens
                if not self.lost:
                    alien.checkCollision(self)
                    #If the aliens have reached the floor, end the game in defeat
                    if alien.y > height:
                        self.end_state(False)
                        break
                        
            #Update the existing rockets
            if not self.lost:
                #Render the rockets in each array with a fixed speed.
                for rocket in self.rockets:
                    rocket.draw(20)
                for rocket in self.enemyRockets:
                    rocket.draw(-2) 
                    
            #Flip the display and iterate the game clock    
            pygame.display.flip()
            self.clock.tick(60)

In [3]:
#Auxillary classes for the Space invaders game mode
class Generator:
    def __init__(self, game):
        #Reset aliens
        game.aliens = []
        #Set attribute information for each alien including position, spacing and type
        width = 50
        margin = 60
        alienType = 4
        #Initialise iterables
        row = 0
        row_index = 0
        #Cycle through as many times as the width, heigh and margin allow, making new aliens
        for y in range(0, int(game.height/2 - width), width):
            row_index = 0
            for x in range(margin, game.width - margin, width):
                game.aliens.append(Alien(game, x, y, 30,
                                         "alien" + str(alienType) + ".png", row, row_index, alienType))
                row_index += 1
            #Use a new alien type for each row
            row += 1
            if alienType > 1:
                alienType -= 1
                
class Alien:
    def __init__(self, game, x, y, size, path, row, row_index, alien_type):
        #Position
        self.x = x
        self.y = y
        #Game reference
        self.game = game
        #Position and type reference
        self.row = row
        self.row_index = row_index
        self.alien_type = alien_type
        
        #Misc
        self.size = size
        self.alienSprites = pygame.sprite.Group()
        self.sprite = Block((130, 240, 120), self.x, self.y, path)
        self.alienSprites.add(self.sprite)
        
        #Initial move direction (denoting right)
        self.moveDirection = 1
        
        
    def draw(self, game):
        for sprite in self.alienSprites:
            #Update horizontal position
            sprite.velocity[0] = game.enemySpeed
            if self.moveDirection > 0:
                sprite.position[0] = sprite.position[0] + sprite.velocity[0]
            else:
                sprite.position[0] = sprite.position[0] - sprite.velocity[0]
            
            #If this is the end alien, and it reaches the side, call drop and reverse for all aliens
            if sprite.position[0] >= game.width-30 and self.moveDirection == 1
            or sprite.position[0] <= 5 and self.moveDirection == -1:
                self.move_direction = -self.move_direction

            for alien in game.aliens:
                alien.drop_and_reverse(self.moveDirection)
            
            #Update sprite position
            sprite.position[1] = sprite.position[1] + sprite.velocity[1]
            sprite.rect.x = sprite.position[0]
            sprite.rect.y = sprite.position[1]
            game.screen.blit(sprite.image, sprite.rect)
    
    def fire(self, game):
        for sprite in self.alienSprites:
            game.enemyRockets.append(Rocket(self.game, sprite.rect.x+16, sprite.rect.y))
        
    def drop_and_reverse(self, newDir):
        #Only update move direction for those who didn't detect the turn
        if self.moveDirection != newDir:
            self.moveDirection = newDir
        #Drop and update sprite position
        for sprite in self.alienSprites:
            sprite.position[1] = sprite.position[1] + 5
            
    def checkCollision(self, game):
        for rocket in game.rockets:
            if rocket.y > game.height or rocket.y < 0:
                game.rockets.remove(rocket)
                
        for sprite in self.alienSprites:
            for rocket in game.rockets:
                if(rocket.x < sprite.rect.x + self.size and
                  rocket.x > sprite.rect.x - self.size and
                  rocket.y < sprite.rect.y + self.size and
                  rocket.y > sprite.rect.y - self.size):
                    game.rockets.remove(rocket)
                    game.aliens.remove(self)
                    game.player.score += 1
                    if len(game.aliens) < 33:
                        game.enemySpeed = 0.25
                    if len(game.aliens) < 22:
                        game.enemySpeed = 0.5
                    if len(game.aliens) < 11:
                        game.enemySpeed = 0.75
                    if len(game.aliens) < 2:
                        game.enemySpeed = 2
                        
#Class for the rocket for both player and alien.
class Rocket:
    def __init__(self, game, x, y):
        self.x = x
        self.y = y
        self.game = game
    
    def draw(self, offset):        
        pygame.draw.rect(self.game.screen, (255, 0, 0),
                        pygame.Rect(self.x, self.y, 2, 4))
        self.y-=offset

#Default container for objects using sprites
class Block(pygame.sprite.Sprite):
    def __init__(self, color, width, height, path):
        super().__init__()
        # Load the image
        self.image = pygame.image.load(path).convert()
        # Set our transparent color
        self.image.set_colorkey((0,0,0))
        self.rect = self.image.get_rect()
        self.rect.x = width
        self.rect.y = height
        self.position = [width, height]
        self.velocity = [0, 0]
        
    def scale(self, scaleFactor):
        self.image = pygame.transform.scale(self.image, (int(self.rect.x/scaleFactor),
                                                         int(self.rect.y/scaleFactor)))
        
        
class Invaders_Player: 
    def __init__(self, game, x, y):
        self.x = x
        self.y = y
        self.size = 30
        self.game = game
        self.playerSprites = pygame.sprite.Group()
        self.sprite = Block((130, 240, 120), self.x, self.y, "paddle.png")
        self.playerSprites.add(self.sprite)
        self.fire_rate = 3.0
        self.score = 0
        
    def draw(self, game):
        self.playerSprites.draw(game.screen)
        if self.fire_rate != 0.0:
            self.fire_rate -=0.1
            if self.fire_rate < 0.0:
                self.fire_rate = 0.0
                
    def checkCollision(self, game):
        for rocket in game.enemyRockets:
            if rocket.y > game.height or rocket.y < 0:
                game.enemyRockets.remove(rocket)
            for sprite in self.playerSprites:
                if(rocket.x < sprite.rect.x + self.size and
                      rocket.x > sprite.rect.x - self.size and
                      rocket.y < sprite.rect.y + self.size and
                      rocket.y > sprite.rect.y - self.size):
                        game.enemyRockets.remove(rocket)
                        game.lost = True

In [8]:
#Game menu class controlling the functionality of the entire framework
class Game_Menu:

    def __init__(self, width, height):
        self.screen = None
        self.menu = None
        self.surface = None
        self.game = None
        self.icon_surface = None
    
        pygame.display.init()
        
        self.surface = pygame.display.set_mode((800, 600))
        self.menu = pygame_menu.Menu(400, 550, 'Welcome',
                               theme=pygame_menu.themes.THEME_BLUE)

        self.menu.add_selector('Difficulty :', [('Hard', 1), ('Easy', 2)], onchange=self.set_difficulty)
        self.menu.add_button('Space Invaders', self.start_space_invaders)
        self.menu.add_button('Space Invaders - Training', self.start_space_invaders_training)
        self.menu.add_button('Asteroids', self.start_asteroids)
        self.menu.add_button('Quit', pygame_menu.events.EXIT)

        self.icon_surface = pygame.image.load('Masterslogo.png')
        pygame.display.set_icon(self.icon_surface)
        pygame.display.set_caption("Master's Project")
        
        self.menu.mainloop(self.surface)

    def set_difficulty(self, value, difficulty):
        # Do the job here !
        pass
    def start_asteroids(self):
        self.game = Asteroids(800, 600, self.screen, self.surface, False)
        pass
    def start_space_invaders(self):
        pygame.display.init()
        self.game = Space_Invaders(700, 600, self.screen, self.surface, False)
        pass
    def start_space_invaders_training(self):
        training_game = Space_Invaders(700, 600, screen, surface, True)
        #Self.game is env
        self.game = Custom_OpenAI_Env(700, 600, self.screen, self.surface, training_game)
        
        #Initialise state/action arrays
        states = env.observation_space.shape
        actions = env.action_space.n
        #Initialise the DRL model
        model = self.build_model(states, actions)
        model.summary()

        #Initialise the DQN agent
        dqn = build_agent(model, actions)
        dqn.compile(Adam(lr=1e-3), metrics=['mae'])
        
        #Fit with openAI gym
        dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)
        
        #Test with scores
        scores = dqn.test(env, nb_episodes=100, visualize=False)
        print(np.mean(scores.history['episode_reward']))

        #Visualize the testing
        #_ = dqn.test(env, nb_episodes=15, visualize=True)

    def save(self, model, name='default-model'):
        #was dqn now model
        model.save_weights(name + '.h5f', overwrite=True)

    def start_asteroids_training(self):
        training_game = Asteroids(800, 600, screen, surface, True)
        self.game = Custom_OpenAI_Env(800, 600, self.screen, self.surface, training_game)
        pass
    
    def build_model(self, states, actions):
      model = Sequential()    
      model.add(Dense(24, activation='relu', input_shape=states))
      model.add(Dense(24, activation='relu'))
      model.add(Dense(actions, activation='linear'))
      return model

    #Function to create DQN model 
    def build_agent(model, actions):
        policy = BoltzmannQPolicy()
        memory = SequentialMemory(limit=50000, window_length=1)
        dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                      nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
        return dqn


#training_game = Space_Invaders(700, 600, screen, surface, True)
#env = Custom_OpenAI_Env(700, 600, self.screen, self.surface, training_game)


#states = env.observation_space.shape
#actions = env.action_space.n


In [9]:
class Asteroids(Training_Game):
    
    def __init__(self, width, height, screen, surface, mode):
        self.gameDisplay = pygame.display.set_mode((width, height))
        self.timer = pygame.time.Clock()
        # Init variables
        self.gameState = "Playing"
        self.player_pieces = []

        self.next_level_delay = 0
        self.bullet_capacity = 4
        self.bullets = []
        self.asteroids = []
        self.stage = 3
        self.score = 0
        self.intensity = 0
        self.player = Asteroid_Player(display_width / 2, display_height / 2)
        self.saucer = Saucer()

        # Main loop
        while gameState != "Exit":
            # User inputs
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    gameState = "Exit"
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        player.thrust = True
                    if event.key == pygame.K_LEFT:
                        player.rtspd = -player_max_rtspd
                    if event.key == pygame.K_RIGHT:
                        player.rtspd = player_max_rtspd
                    if event.key == pygame.K_SPACE and len(bullets) < bullet_capacity:
                        bullets.append(Bullet(player.x, player.y, player.dir))
                    if gameState == "Game Over":
                        if event.key == pygame.K_r:
                            #GO back to menu here
                            gameState = "Exit"
                if event.type == pygame.KEYUP:
                    if event.key == pygame.K_UP:
                        player.thrust = False
                    if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                        player.rtspd = 0

            # Update player
            player.updatePlayer()

            # Reset display
            self.gameDisplay.fill(black)

            # Check for collision w/ asteroid
            for a in asteroids:
                a.updateAsteroid(self.gameDisplay)
                if player.player_state != "Died":
                    if isColliding(player.x, player.y, a.x, a.y, a.size):
                        player_pieces = np.concatenate([player_pieces, player.killPlayer()], axis = None)
                        gameState = "Game Over"

                        # Split asteroid
                        if a.t > 10:
                            asteroids.append(Asteroid(a.x, a.y, a.t-10))
                            asteroids.append(Asteroid(a.x, a.y, a.t-10))
                            score += 20 if a.t == 30 else 50
                        else:
                            score +=100
                        asteroids.remove(a)
            # Update ship fragments
            for f in player_pieces:
                f.updateDeadPlayer(self.gameDisplay)
                if f.x > display_width or f.x < 0 or f.y > display_height or f.y < 0:
                    player_pieces.tolist().remove(f)

            # Check for end of stage
            if len(asteroids) == 0 and saucer.state == "Dead":
                if next_level_delay < 30:
                    next_level_delay += 1
                else:
                    stage += 1
                    intensity = 0
                    # Spawn asteroid away of center
                    for i in range(stage):
                        xTo = display_width / 2
                        yTo = display_height / 2
                        while xTo - display_width / 2 < display_width / 4 and yTo - display_height / 2 < display_height / 4:
                            xTo = random.randrange(0, display_width)
                            yTo = random.randrange(0, display_height)
                        asteroids.append(Asteroid(xTo, yTo, 30))
                    next_level_delay = 0

            # Update intensity
            if intensity < stage * 450:
                intensity += 1

            # Saucer
            if saucer.state == "Dead":
                if random.randint(0, 6000) <= (intensity * 2) / (stage * 9) and next_level_delay == 0:
                    saucer.createSaucer()
                    # Only small saucers >40000
                    if score >= 40000:
                        saucer.type = "Small"
            else:
                # Set saucer targer dir
                acc = small_saucer_accuracy * 4 / stage
                saucer.bdir = math.degrees(math.atan2(-saucer.y + player.y, -saucer.x + player.x) + math.radians(random.uniform(acc, -acc)))

                saucer.updateSaucer()
                saucer.drawSaucer(self.gameDisplay)

                # Check for collision w/ asteroid
                for a in asteroids:
                    if isColliding(saucer.x, saucer.y, a.x, a.y, a.size + saucer.size):
                        # Set saucer state
                        saucer.state = "Dead"
                        # Split asteroid                        
                        if a.t > 10:
                            asteroids.append(Asteroid(a.x, a.y, a.t-10))
                            asteroids.append(Asteroid(a.x, a.y, a.t-10))
                            score += 20 if a.t == 30 else 50
                        else:
                            score +=100
                        asteroids.remove(a)

                # Check for collision w/ bullet
                for b in bullets:
                    if isColliding(b.x, b.y, saucer.x, saucer.y, saucer.size):
                        # Add points
                        if saucer.type == "Large":
                            score += 200
                        else:
                            score += 1000
                        # Set saucer state
                        saucer.state = "Dead"
                        # Remove bullet
                        bullets.remove(b)

                # Check collision w/ player
                if isColliding(saucer.x, saucer.y, player.x, player.y, saucer.size):
                    if player.player_state != "Died":
                        # Create ship fragments
                        player_pieces.append(deadPlayer(player.x, player.y, 5 * 10 / (2 * math.cos(math.atan(1 / 3)))))
                        player_pieces.append(deadPlayer(player.x, player.y, 5 * 10 / (2 * math.cos(math.atan(1 / 3)))))
                        player_pieces.append(deadPlayer(player.x, player.y, 10))

                        # Kill player
                        player.player_state = "Died"
                        player.killPlayer()
   
                        gameState = "Game Over"

                # Saucer's bullets
                for b in saucer.bullets:
                    # Update bullets
                    b.updateBullet(self.gameDisplay)

                    # Check for collision w/ asteroids
                    for a in asteroids:
                        if isColliding(b.x, b.y, a.x, a.y, a.size):
                            if a.t > 10:
                                asteroids.append(Asteroid(a.x, a.y, a.t-10))
                                asteroids.append(Asteroid(a.x, a.y, a.t-10))
                                score += 20 if a.t == 30 else 50
                            else:
                                score +=100
                            asteroids.remove(a)
                            saucer.bullets.remove(b)
                            break

                    # Check for collision w/ player
                    if isColliding(player.x, player.y, b.x, b.y, 5):
                        if player.player_state != "Died":
                            # Create ship fragments
                            player_pieces.append(deadPlayer(player.x, player.y, 5 * 10 / (2 * math.cos(math.atan(1 / 3)))))
                            player_pieces.append(deadPlayer(player.x, player.y, 5 * 10 / (2 * math.cos(math.atan(1 / 3)))))
                            player_pieces.append(deadPlayer(player.x, player.y, 10))

                            # Kill player
                            player.player_state = "Died"
                            player.killPlayer()
                            
                            gameState = "Game Over"

                            # Remove bullet
                            saucer.bullets.remove(b)

                    if b.life <= 0:
                        try:
                            saucer.bullets.remove(b)
                        except ValueError:
                            continue

            # Bullets
            for b in bullets:
                # Update bullets
                b.updateBullet(self.gameDisplay)

                # Check for bullets collide w/ asteroid
                for a in asteroids:
                    if b.x > a.x - a.size and b.x < a.x + a.size and b.y > a.y - a.size and b.y < a.y + a.size:
                        # Split asteroid
                        if a.t > 10:
                            asteroids.append(Asteroid(a.x, a.y, a.t-10))
                            asteroids.append(Asteroid(a.x, a.y, a.t-10))
                            score += 20 if a.t == 30 else 50
                        else:
                            score +=100
                        asteroids.remove(a)
                        bullets.remove(b)

                        break

                # Destroying bullets
                if b.life <= 0:
                    try:
                        bullets.remove(b)
                    except ValueError:
                        continue

            # Draw player
            if gameState != "Game Over":
                if not player.player_state == "Died":
                    player.drawPlayer(self.gameDisplay)
            else:
                self.drawText("Game Over", white, display_width / 2, display_height / 2, 100)
                self.drawText("Press \"R\" to Return to the menu.", white, display_width / 2, display_height / 2 + 100, 50)

            # Draw score
            self.drawText(str(score), white, 60, 20, 40, False)
            # Update screen
            pygame.display.update()

            # Tick fps
            timer.tick(30)
    # Create function to draw texts
    def drawText(self, msg, color, x, y, s, center=True):
        screen_text = pygame.font.SysFont("Calibri", s).render(msg, True, color)
        if center:
            rect = screen_text.get_rect()
            rect.center = (x, y)
        else:
            rect = (x, y)
        self.gameDisplay.blit(screen_text, rect)


In [ ]:
#Auxillary classes for the Asteroids game mode
# Initialize constants
white = (255, 255, 255)
red = (255, 0, 0)
black = (0, 0, 0)

#player_size = 10
fd_fric = 0.5
bd_fric = 0.1
player_max_speed = 20
player_max_rtspd = 10
bullet_speed = 15
saucer_speed = 5
small_saucer_accuracy = 10

# Create funtion to chek for collision
def isColliding(x, y, xTo, yTo, size):
    if x > xTo - size and x < xTo + size and y > yTo - size and y < yTo + size:
        return True
    return False


# Create class asteroid
class Asteroid:
    def __init__(self, x, y, t):
        self.x = x
        self.y = y
        self.size = t
        self.t = t

        # Make random speed and direction
        self.speed = random.uniform(1, (40 - self.size) * 4 / 15)
        self.dir = random.randrange(0, 360) * math.pi / 180

        # Make random asteroid sprites
        full_circle = random.uniform(18, 36)
        dist = random.uniform(self.size / 2, self.size)
        self.vertices = []
        while full_circle < 360:
            self.vertices.append([dist, full_circle])
            dist = random.uniform(self.size / 2, self.size)
            full_circle += random.uniform(18, 36)

    def updateAsteroid(self, gameDisplay):
        # Move asteroid
        self.x += self.speed * math.cos(self.dir)
        self.y += self.speed * math.sin(self.dir)

        # Check for wrapping
        if self.x > display_width:
            self.x = 0
        elif self.x < 0:
            self.x = display_width
        elif self.y > display_height:
            self.y = 0
        elif self.y < 0:
            self.y = display_height

        # Draw asteroid
        for v in range(len(self.vertices)):
            if v == len(self.vertices) - 1:
                next_v = self.vertices[0]
            else:
                next_v = self.vertices[v + 1]
            this_v = self.vertices[v]
            pygame.draw.line(gameDisplay, white, (self.x + this_v[0] * math.cos(this_v[1] * math.pi / 180),
                                                  self.y + this_v[0] * math.sin(this_v[1] * math.pi / 180)),
                             (self.x + next_v[0] * math.cos(next_v[1] * math.pi / 180),
                              self.y + next_v[0] * math.sin(next_v[1] * math.pi / 180)))


# Create class bullet
class Bullet:
    def __init__(self, x, y, direction):
        self.x = x
        self.y = y
        self.dir = direction
        self.life = 30

    def updateBullet(self, gameDisplay):
        # Moving
        self.x += bullet_speed * math.cos(self.dir * math.pi / 180)
        self.y += bullet_speed * math.sin(self.dir * math.pi / 180)

        # Drawing
        pygame.draw.circle(gameDisplay, white, (int(self.x), int(self.y)), 3)

        # Wrapping
        if self.x > display_width:
            self.x = 0
        elif self.x < 0:
            self.x = display_width
        elif self.y > display_height:
            self.y = 0
        elif self.y < 0:
            self.y = display_height
        self.life -= 1


# Create class saucer
class Saucer:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.state = "Dead"
        self.type = "Large"
        self.dirchoice = ()
        self.bullets = []
        self.cd = 0
        self.bdir = 0
        self.soundDelay = 0

    def updateSaucer(self):
        # Move player
        self.x += saucer_speed * math.cos(self.dir * math.pi / 180)
        self.y += saucer_speed * math.sin(self.dir * math.pi / 180)

        # Choose random direction
        if random.randrange(0, 100) == 1:
            self.dir = random.choice(self.dirchoice)

        # Wrapping
        if self.y < 0:
            self.y = display_height
        elif self.y > display_height:
            self.y = 0
        if self.x < 0 or self.x > display_width:
            self.state = "Dead"

        # Shooting
        if self.type == "Large":
            self.bdir = random.randint(0, 360)
        if self.cd == 0:
            self.bullets.append(Bullet(self.x, self.y, self.bdir))
            self.cd = 30
        else:
            self.cd -= 1

    def createSaucer(self):
        # Create saucer
        # Set state
        self.state = "Alive"

        # Set random position
        self.x = random.choice((0, display_width))
        self.y = random.randint(0, display_height)

        # Set random type
        if random.randint(0, 1) == 0:
            self.type = "Large"
            self.size = 20
        else:
            self.type = "Small"
            self.size = 10

        # Create random direction
        if self.x == 0:
            self.dir = 0
            self.dirchoice = (0, 45, -45)
        else:
            self.dir = 180
            self.dirchoice = (180, 135, -135)

        # Reset bullet cooldown
        self.cd = 0

    def drawSaucer(self, gameDisplay):
        # Draw saucer
        pygame.draw.polygon(gameDisplay, white,
                            ((self.x + self.size, self.y),
                             (self.x + self.size / 2, self.y + self.size / 3),
                             (self.x - self.size / 2, self.y + self.size / 3),
                             (self.x - self.size, self.y),
                             (self.x - self.size / 2, self.y - self.size / 3),
                             (self.x + self.size / 2, self.y - self.size / 3)), 1)
        pygame.draw.line(gameDisplay, white,
                         (self.x - self.size, self.y),
                         (self.x + self.size, self.y))
        pygame.draw.polygon(gameDisplay, white,
                            ((self.x - self.size / 2, self.y - self.size / 3),
                             (self.x - self.size / 3, self.y - 2 * self.size / 3),
                             (self.x + self.size / 3, self.y - 2 * self.size / 3),
                             (self.x + self.size / 2, self.y - self.size / 3)), 1)


# Create class for shattered ship
class deadPlayer:
    def __init__(self, x, y, l):
        self.angle = random.randrange(0, 360) * math.pi / 180
        self.dir = random.randrange(0, 360) * math.pi / 180
        self.rtspd = random.uniform(-0.25, 0.25)
        self.x = x
        self.y = y
        self.lenght = l
        self.speed = random.randint(2, 8)

    def updateDeadPlayer(self, gameDisplay):
        pygame.draw.line(gameDisplay, white,
                         (self.x + self.lenght * math.cos(self.angle) / 2,
                          self.y + self.lenght * math.sin(self.angle) / 2),
                         (self.x - self.lenght * math.cos(self.angle) / 2,
                          self.y - self.lenght * math.sin(self.angle) / 2))
        self.angle += self.rtspd
        self.x += self.speed * math.cos(self.dir)
        self.y += self.speed * math.sin(self.dir)


# Class player
class Asteroid_Player:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.hspeed = 0
        self.vspeed = 0
        self.dir = -90
        self.rtspd = 0
        self.thrust = False
        self.player_state = "Alive"

    def updatePlayer(self):
        # Move player
        speed = math.sqrt(self.hspeed**2 + self.vspeed**2)
        if self.thrust:
            if speed + fd_fric < player_max_speed:
                self.hspeed += fd_fric * math.cos(self.dir * math.pi / 180)
                self.vspeed += fd_fric * math.sin(self.dir * math.pi / 180)
            else:
                self.hspeed = player_max_speed * math.cos(self.dir * math.pi / 180)
                self.vspeed = player_max_speed * math.sin(self.dir * math.pi / 180)
        else:
            if speed - bd_fric > 0:
                change_in_hspeed = (bd_fric * math.cos(self.vspeed / self.hspeed))
                change_in_vspeed = (bd_fric * math.sin(self.vspeed / self.hspeed))
                if self.hspeed != 0:
                    if change_in_hspeed / abs(change_in_hspeed) == self.hspeed / abs(self.hspeed):
                        self.hspeed -= change_in_hspeed
                    else:
                        self.hspeed += change_in_hspeed
                if self.vspeed != 0:
                    if change_in_vspeed / abs(change_in_vspeed) == self.vspeed / abs(self.vspeed):
                        self.vspeed -= change_in_vspeed
                    else:
                        self.vspeed += change_in_vspeed
            else:
                self.hspeed = 0
                self.vspeed = 0
        self.x += self.hspeed
        self.y += self.vspeed

        # Check for wrapping
        if self.x > display_width:
            self.x = 0
        elif self.x < 0:
            self.x = display_width
        elif self.y > display_height:
            self.y = 0
        elif self.y < 0:
            self.y = display_height

        # Rotate player
        self.dir += self.rtspd

    def drawPlayer(self, gameDisplay):
        a = math.radians(self.dir)
        x = self.x
        y = self.y
        s = 10
        t = self.thrust
        # Draw player
        for i in range(0,2):
            pygame.draw.line(gameDisplay, white,
                (x - (s * math.sqrt(130) / 12) * math.cos(math.atan(7 / 9) + a),
                y - (s * math.sqrt(130) / 12) * math.sin(math.atan(7 / 9) + a)),
                (x + s * math.cos(a), y + s * math.sin(a)))
        
        pygame.draw.line(gameDisplay, white,
                         (x - (s * math.sqrt(130) / 12) * math.cos(math.atan(7 / 9) - a),
                          y + (s * math.sqrt(130) / 12) * math.sin(math.atan(7 / 9) - a)),
                         (x + s * math.cos(a), y + s * math.sin(a)))

        pygame.draw.line(gameDisplay, white,
                         (x - (s * math.sqrt(2) / 2) * math.cos(a + math.pi / 4),
                          y - (s * math.sqrt(2) / 2) * math.sin(a + math.pi / 4)),
                         (x - (s * math.sqrt(2) / 2) * math.cos(-a + math.pi / 4),
                          y + (s * math.sqrt(2) / 2) * math.sin(-a + math.pi / 4)))
        if t:
            pygame.draw.line(gameDisplay, white,
                             (x - s * math.cos(a),
                              y - s * math.sin(a)),
                             (x - (s * math.sqrt(5) / 4) * math.cos(a + math.pi / 6),
                              y - (s * math.sqrt(5) / 4) * math.sin(a + math.pi / 6)))
            pygame.draw.line(gameDisplay, white,
                             (x - s * math.cos(-a),
                              y + s * math.sin(-a)),
                             (x - (s * math.sqrt(5) / 4) * math.cos(-a + math.pi / 6),
                              y + (s * math.sqrt(5) / 4) * math.sin(-a + math.pi / 6)))

    def killPlayer(self):
        
        player_debris = []
        player_debris.append(deadPlayer(self.x, self.y, 5 * 10 / (2 * math.cos(math.atan(1 / 3)))))
        player_debris.append(deadPlayer(self.x, self.y, 5 * 10 / (2 * math.cos(math.atan(1 / 3)))))
        player_debris.append(deadPlayer(self.x, self.y, 10))
        # Kill player
        self.player_state = "Died"
        return player_debris

In [ ]:
Game = Game_Menu(800,600)

called generator
length potential shooters:  11
surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 

length potential shooters:  11
surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 

surface information: 
dimensions:  2 
Shape:  (700, 600) 



In [ ]:
#Implement DQN agent

#Env is custom openAI env
#TODO: add refernces to screen and surface to be passed through here. 
#Add this framework into the menu screen then test to make drl agent simply for space invaders.
training_game = Space_Invaders(700, 600, screen, surface, True)
env = Custom_OpenAI_Env(700, 600, self.screen, self.surface, training_game)


states = env.observation_space.shape
actions = env.action_space.n

def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model


model = build_model(states, actions)
model.summary()


#Function to create DQN model 
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))


#_ = dqn.test(env, nb_episodes=15, visualize=True)


dqn.save_weights('dqn_weights.h5f', overwrite=True)

del model
del dqn
del env

env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

dqn.load_weights('dqn_weights.h5f')

 = dqn.test(env, nb_episodes=5, visualize=True)